In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.environ.get("OPENAI_API_KEY")
neo4j_url = os.environ.get("NEO4J_URL")
neo4j_username = os.environ.get("NEO4J_USERNAME")
neo4j_password = os.environ.get("NEO4J_PASSWORD")

In [5]:
#!unset neo4j_url

In [2]:
neo4j_url

'neo4j+s://456b56ec.databases.neo4j.io'

In [3]:
from langchain.graphs import Neo4jGraph

url = neo4j_url
username =neo4j_username
password = neo4j_password

graph = Neo4jGraph(
    url=url, 
    username=username, 
    password=password
)

In [4]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI

# Set up the GraphCypherQAChain
cypher_chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-0125"),
    qa_llm=ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-0125"),
    validate_cypher=True,
    verbose=True,
    allow_dangerous_requests=True,
    return_intermediate_steps=True
)

# Run the query
### Enhancement Query
query="give me a Articles talking about Mpox in Mexico"
eq = query + "Consider this: for the new, return the url,label,the country, the virus, and the abstract. in the aswer dont consider a prefix of the column."

response =  cypher_chain.invoke({"query": eq})
print(f"Intermediate steps: {response['intermediate_steps']}")
print(f"Final answer: {response['result']}")

/var/folders/gd/2hyknd_n0db256k8n9_snpww0000gn/T/ipykernel_1124/1562747253.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  cypher_llm=ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-0125"),
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [11]:
Query_Cypher = response['intermediate_steps'][0]["query"]

In [12]:
response = graph.query(
    Query_Cypher
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownPropertyKeyWarning} {category: UNRECOGNIZED} {title: The provided property key is not in the database} {description: One of the property names in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing property name is: organization)} {position: line: 2, column: 49, offset: 145} for query: 'MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus{name:"Mpox"})-[:AFFECTS]->(l:Location{name:"Mexico"})\nRETURN a.url, a.label, l.country, a.abstract, a.organization, v.name'


In [13]:
    response

[{'a.url': 'https://www.streetinsider.com/Reuters/Argentina+health+officials+determine+as+false+suspected+case+of+mpox+virus+on+ship/23628564.html',
  'a.label': 'true',
  'l.country': 'MX',
  'a.abstract': 'Failed to extract article: Article `download()` failed with 403 Client Error: Forbidden for url: https://www.streetinsider.com/Reuters/Argentina+health+officials+determine+as+false+suspected+case+of+mpox+virus+on+ship/23628564.html on URL https://www.streetinsider.com/Reuters/Argentina+health+officials+determine+as+false+suspected+case+of+mpox+virus+on+ship/23628564.html',
  'a.organization': None,
  'v.name': 'Mpox'},
 {'a.url': 'https://www.theglobeandmail.com/world/article-ship-in-argentinas-parana-river-is-quarantined-on-suspected-mpox-case/',
  'a.label': 'false',
  'l.country': 'MX',
  'a.abstract': 'Open this photo in gallery: Ships carry grains for export on the Parana River, in Rosario, Argentina, on Aug. 9.Matias Baglietto/Reuters\n\nA ship on Argentina’s Parana River nea

In [8]:
url = response[0]["n"]["url"]
url

'https://www.correiobraziliense.com.br/politica/2024/07/6894080-esquema-das-joias-movimentou-rs-25-mi-bolsonaro-recebeu-dinheiro-vivo-diz-pf.html'

## Scrapping web using beautifull soup

In [9]:
import requests
from bs4 import BeautifulSoup

# Step 1: Send a GET request to the webpage you want to scrape
response = requests.get(url)

# Step 2: Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Step 3: Extract the data you need
# Example: Getting the title of the webpage
title = soup.title.string
print(f'Title of the page: {title}')

# You can extract other elements by their tag, class, id, etc.
# Example: Get all paragraph elements
data_scrapped = ""
paragraphs = soup.find_all('p')
for paragraph in paragraphs:
    data_scrapped+=paragraph.text + "\n"
    print(paragraph.text)

Title of the page: Esquema das joias movimentou R$ 6,8 mi; Bolsonaro recebeu dinheiro vivo, diz PF
Ex-presidente Jair Bolsonaro é indiciado pela PF no caso das joias
Moraes retira sigilo do inquérito das joias e manda PGR se manifestar
DÃª a sua opiniÃ£o! O Correio tem um espaÃ§o na ediÃ§Ã£o impressa para publicar a opiniÃ£o dos leitores pelo e-mail sredat.df@dabr.com.br
RepÃ³rter de polÃ­tica, setorista do Supremo Tribunal Federal (STF). Vencedor do PrÃªmio CNT de Jornalismo, possuÃ­ passagens pelo SBT, Record/R7 e estÃ¡ entre os jornalistas mais influentes do Twitter no Brasil.
Formada em Jornalismo pela UFSM, no RS, onde pesquisou sobre Jornalismo Internacional, focado em AmÃ©rica Latina. JÃ¡ trabalhou com Cultura, Entretenimento, Redes Sociais, GÃªnero e Minorias Sociais.


In [10]:
data_scrapped

'Ex-presidente Jair Bolsonaro é indiciado pela PF no caso das joias\nMoraes retira sigilo do inquérito das joias e manda PGR se manifestar\nDÃª a sua opiniÃ£o! O Correio tem um espaÃ§o na ediÃ§Ã£o impressa para publicar a opiniÃ£o dos leitores pelo e-mail sredat.df@dabr.com.br\nRepÃ³rter de polÃ\xadtica, setorista do Supremo Tribunal Federal (STF). Vencedor do PrÃªmio CNT de Jornalismo, possuÃ\xad passagens pelo SBT, Record/R7 e estÃ¡ entre os jornalistas mais influentes do Twitter no Brasil.\nFormada em Jornalismo pela UFSM, no RS, onde pesquisou sobre Jornalismo Internacional, focado em AmÃ©rica Latina. JÃ¡ trabalhou com Cultura, Entretenimento, Redes Sociais, GÃªnero e Minorias Sociais.\n'

## Check if the new is True o False

In [51]:
## output parser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
from pydantic import BaseModel, Field, model_validator
from langchain_core.output_parsers import PydanticOutputParser

# Define your desired data structure.
class NewChecker(BaseModel):
    new_title: str = Field(description="the new's title")
    reliable: str = Field(description="provide details about the source reliable")
    content: str = Field(description = "The content of the news supported by verifiable evidence")
    score : float = Field(description ="a score between 0 and 1, where 1 means the news is completely true and closer to 0 means it is false")
parser = PydanticOutputParser(pydantic_object=NewChecker)


In [52]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# Define the LLM (using OpenAI as an example)
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

# Define the prompt template
template = """
You are an expert in fact-checking, and your task is to analyze the following news to determine whether it is true or false.
News: {news}

Provide a detailed explanation based on the following questions:
1. Is the source of the news reliable? If possible, provide details about the source.
2. Is the content of the news supported by verifiable evidence or links to credible sources?
4. Finally, based on your analysis, provide your reasoning and conclude with a score between 0 and 1, where 1 means the news is completely true and closer to 0 means it is false.

{format_instructions}

"""

prompt = PromptTemplate(
    template=template,
    input_variables=["news"],
    partial_variables={"format_instructions": parser.with_structured_output()},
)

# Generate the final prompt with the news article
final_prompt = prompt.format(news=data_scrapped)

# Use the LLM to generate a response
response = llm(final_prompt)

# Print the response from the model
print(response)


AttributeError: 'PydanticOutputParser' object has no attribute 'with_structured_output'